In [3]:
import json

In [4]:
import random

In [5]:
# fname = 'full_questions_v2.json'
dataset_split = 'test'
fname = '{split}_questions_v3.json'.format(split=dataset_split)
f = open(fname, 'r')
json_string = ''
for line in f:
    json_string += line
f.close()

In [6]:
questions = json.loads(json_string)

In [7]:
len(questions)

30003

In [8]:
questions[0]

{'question': 'When did Q703956 play in Q1886?',
 'answers': [1992, 1993, 1994, 1995, 2000, 2001],
 'answer_type': 'time',
 'template': 'When did {head} play in {tail}?',
 'entities': ['Q1886', 'Q703956'],
 'times': [],
 'relations': ['P54'],
 'paraphrases': ['When did Maurizio Ganz play in Atalanta Bergamasca Calcio s.p.a.?',
  'When did Maurizio Ganz play in Atalanta B.C.?',
  'When did Maurizio Ganz play in Atalanta Bergamasca Calcio?',
  'When did Maurizio Ganz play in Atalanta?',
  'When did Maurizio Ganz play in Atalanta Bergamo?'],
 'annotation': {'head': 'Q703956', 'tail': 'Q1886'},
 'type': 'simple_time'}

In [9]:
import pickle
aliases_file = '/scratche/home/apoorv/tempqa/data/temporal_big/wd_id_to_aliases.pickle'
aliases = pickle.load(open(aliases_file, 'rb'))

In [18]:
import random
from unidecode import unidecode

def generatePairsForPP(question, max=5):
    template = question['template']
    template.replace('?', '')
    split_template = template.split(' ')
    split_template_no_bracket_tokens = []
    for s in split_template:
        if '{' in s and '}' in s:
            continue
        else:
            split_template_no_bracket_tokens.append(s)
    keywords_not_to_remove = ['When', 'Who']
    candidate_keywords = set(split_template_no_bracket_tokens) - set(keywords_not_to_remove)
    all_pp = []
    original_list = []
    candidate_keywords = list(candidate_keywords)
    random.shuffle(candidate_keywords)
    for paraphrase in question['paraphrases']:
        for keyword in candidate_keywords:
            # need to remove unicode chars for paraphraser input
            all_pp.append(unidecode(paraphrase) + '\t' + keyword) 
            original_list.append(paraphrase)
    pairs = []
    for i in range(len(all_pp)):
        pairs.append((original_list[i], all_pp[i]))
    if len(pairs) <= max:
        return pairs
    else:
        return random.sample(pairs, max)
    

In [19]:
pairs = generatePairsForPP(questions[0])

In [20]:
pairs

[('When did Maurizio Ganz play in Atalanta?',
  'When did Maurizio Ganz play in Atalanta?\tin'),
 ('When did Maurizio Ganz play in Atalanta Bergamo?',
  'When did Maurizio Ganz play in Atalanta Bergamo?\tdid'),
 ('When did Maurizio Ganz play in Atalanta B.C.?',
  'When did Maurizio Ganz play in Atalanta B.C.?\tdid'),
 ('When did Maurizio Ganz play in Atalanta Bergamasca Calcio s.p.a.?',
  'When did Maurizio Ganz play in Atalanta Bergamasca Calcio s.p.a.?\tplay'),
 ('When did Maurizio Ganz play in Atalanta?',
  'When did Maurizio Ganz play in Atalanta?\tdid')]

In [21]:
from tqdm import tqdm
all_triples = []
for i,q in enumerate(tqdm(questions)):
    pairs = generatePairsForPP(q)
    triples = []
    for p in pairs:
        triples.append([i, p[0], p[1]])
    all_triples += triples

100%|██████████| 30003/30003 [00:03<00:00, 9129.26it/s] 


In [ ]:
len(all_triples)

In [23]:
import pickle
pp_triples_file = 'pp_triples_v3.pickle'
pickle.dump(all_triples, open(pp_triples_file, 'wb'))

In [24]:
# def getParaphrases(lines):
#     paraphrases = []
#     f = open('temp.txt', 'w')
#     for line in lines:
#         f.write(line + '\n')
#     f.close()
#     ! cat 'temp.txt' | ./paraphrase.sh --use-cpu > 'temp2.txt'
#     f = open('temp2.txt', 'r')
#     for line in f:
#         paraphrases.append(line.strip())
#     return paraphrases

def getParaphrases(triples):
    question_ids = [x[0] for x in triples]
    original_questions = [x[1] for x in triples]
    pp_lines = [x[2] for x in triples]
    f = open('temp.txt', 'w')
    for line in pp_lines:
        f.write(line + '\n')
    f.close()
    ! cat 'temp.txt' | ./paraphrase.sh --use-cpu > 'temp2.txt'
    f = open('temp2.txt', 'r')
    paraphrases = []
    for line in f:
        paraphrases.append(line.strip())
    paraphrase_triples = []
    for i in range(len(pp_lines)):
        paraphrase_triples.append([question_ids[i], original_questions[i], paraphrases[i]])
    return paraphrase_triples

In [25]:
x = getParaphrases(all_triples[48777:48777+10])

/home/apoorv/anaconda3/envs/kge/bin/python3: Error while finding module specification for 'sockeye.translate' (ModuleNotFoundError: No module named 'sockeye')
Traceback (most recent call last):
  File "./detok.py", line 6, in <module>
    from mosestokenizer import MosesDetokenizer
ModuleNotFoundError: No module named 'mosestokenizer'
Traceback (most recent call last):
  File "./custom_constraints.py", line 5, in <module>
    from subword_nmt.apply_bpe import *
ModuleNotFoundError: No module named 'subword_nmt'


IndexError: list index out of range

In [213]:
x

[[10000,
  'When did Vlado Šmit play their first game?',
  'When was Vlado Smit playing his first game?'],
 [10000,
  'When did Vlado Šmit play their first game?',
  'When was Vlado Smit playing their first game?'],
 [10000,
  'When did Vlado Šmit play their first game?',
  'When was Vlado Smit playing their first game?'],
 [10000,
  'When did Vlado Šmit play their first game?',
  'When was Vlado Smit playing their first game?'],
 [10001,
  'When did Vlado Šmit play their first game for Bologna Football Club 1909?',
  'When was Vlado Smit in Bologna Football Club in 1909?'],
 [10001,
  'When did Vlado Šmit play their first game for Bologna F.C. 1909?',
  'When was Vlado Smit playing their first game in Bologna F.C. in 1909?'],
 [10001,
  'When did Vlado Šmit play their first game for Bologna FC 1909?',
  'When was Vlado Smit playing their first game at Bologna FC in 1909?'],
 [10001,
  'When did Vlado Šmit play their first game for Bologna Football Club 1909?',
  'When was Vlado Smit p

In [142]:
from collections import defaultdict
paraphrase_dict = defaultdict(list)

for pair in x:
    original = pair[0]
    pp = pair[1]
    if pp not in paraphrase_dict[original]:
        paraphrase_dict[original].append(pp)
    

In [143]:
len(paraphrase_dict)

242

In [144]:
for k, v in paraphrase_dict.items():
    pass
#     print(k, v)

In [145]:
output_list = []
for k, v in paraphrase_dict.items():
    item = {}
    item['original'] = k
    item['paraphrases'] = v
    output_list.append(item)

In [146]:
output_list[10]

{'original': 'When did Gunnar Gren play their last game?',
 'paraphrases': ['When was Gunnar Gren playing their last game?']}

In [149]:
import json
output_json = ','.join([json.dumps(item) for item in output_list])
output_json = '[' + output_json + ']'

In [150]:
len(output_json)

51634

In [151]:
fname = 'paraphrases.json'
f = open(fname, 'w')
f.write(output_json)
f.close()

In [171]:
output_list[144]

{'original': 'When did Pavel Andreyevich Taran receive Order of the Red Banner?',
 'paraphrases': ["When was Pavel Andreyevich Taran given the Red Banner's Order?",
  "When did Pavel Andreyevich Taran get the Red Banner's Order?"]}

In [166]:
len(output_list)

242

In [214]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())


Number of processors:  56
